In [1]:
!pip install requests

In [ ]:
import requests
import json
# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

print(type(data))
print(len(data))
print(json.dumps(data[0], indent=2))

In [4]:
def json_print(data):
    print(json.dumps(data, indent=2))

json_print(data)

[
  {
    "Category": "SCIENCE",
    "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
    "Answer": "Liver"
  },
  {
    "Category": "ANIMALS",
    "Question": "It's the only living mammal in the order Proboseidea",
    "Answer": "Elephant"
  },
  {
    "Category": "ANIMALS",
    "Question": "The gavial looks very much like a crocodile except for this bodily feature",
    "Answer": "the nose or snout"
  },
  {
    "Category": "ANIMALS",
    "Question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
    "Answer": "Antelope"
  },
  {
    "Category": "ANIMALS",
    "Question": "Heaviest of all poisonous snakes is this North American rattlesnake",
    "Answer": "the diamondback rattler"
  },
  {
    "Category": "SCIENCE",
    "Question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
    "Answer": "species"
  },
  {
    "Category": "SCIENCE",
   

In [6]:
!pip install weaviate-client

     |████████████████████████████████| 120 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 223 kB 16.8 MB/s eta 0:00:01


In [10]:
#Intialize weaviate - Vector DB
import weaviate
from weaviate import EmbeddedOptions
import os

#start a weaviate instance
client = weaviate.Client(embedded_options=EmbeddedOptions(), additional_headers={
    "X-OpenAI-API-Key": "XXXXXXXXXXXXXXXXXXXXX"})



Binary /Users/a8h1/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.0/weaviate-v1.23.0-Darwin-all.zip
Started /Users/a8h1/.cache/weaviate-embedded: process ID 15033


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-29T19:26:34-08:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-29T19:26:34-08:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-01-29T19:26:34-08:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-01-29T19:26:34-08:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-01-29T19:26:34-08:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

{"action":"read_disk_use","level":"warning","msg":"disk usage currently at 97.84%, threshold set to 80.00%","path":"/Users/a8h1/.local/share/weaviate","time":"2024-01-29T19:26:34-08:00"}
{"action":"set_shard_read_only","level":"warning","msg":"Set READONLY, disk usage currently at 97.84%, threshold set to 90.00%","path":"/Users/a8h1/.local/share/weaviate","time":"2024-01-29T19:26:34-08:00"}


In [11]:
json_print(client.get_meta())

{
  "hostname": "http://127.0.0.1:8079",
  "modules": {
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/rerank/",
      "name": "Reranker - Cohere"
    },
    "text2vec-cohere": {
      "documentationHref": "https://docs.cohere.ai/embedding-wiki/",
      "name": "Cohere Module"
    },
    "text2vec-huggingface": {
      "documentationHref": "https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task",
      "name": "Hugging Face Module"
    },
    "text2vec-openai": {
      "documentationHref": "https://platform.openai.com/docs/guides/embeddings/what-are-embeddings",
      "nam

In [ ]:
#Delete the schema if it already exists
if client.schema.exists("Question"):
    client.schema.delete_class("Question")


In [12]:
#Create the schema that will house our data
class_obj = {
    "class" : "Question",
    "vectorizer": "text2vec-openai",
}

#use the object above the schema
client.schema.create_class(class_obj)


{"level":"info","msg":"Created shard question_b3BFRlma9hfm in 2.843542ms","time":"2024-01-29T19:31:08-08:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-29T19:31:08-08:00","took":899083}


In [14]:
with client.batch.configure() as batch:
    for i, d in enumerate(data):
        print(f"importing questions : {i+1}")

        properties = {
            "answer" : d["Answer"],
            "questions": d["Question"],
            "category": d["Category"]
        }

        batch.add_data_object(
            data_object = properties,
            class_name = "Question"
        )

importing questions : 1
importing questions : 2
importing questions : 3
importing questions : 4
importing questions : 5
importing questions : 6
importing questions : 7
importing questions : 8
importing questions : 9
importing questions : 10


In [15]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}


In [17]:
#Extract 3 questions and answers

json_print(client.query.get("Question", ["questions", "answer"]).with_limit(3).do())

{
  "data": {
    "Get": {
      "Question": [
        {
          "answer": "the diamondback rattler",
          "questions": "Heaviest of all poisonous snakes is this North American rattlesnake"
        },
        {
          "answer": "the nose or snout",
          "questions": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "answer": "Elephant",
          "questions": "It's the only living mammal in the order Proboseidea"
        }
      ]
    }
  }
}
